Задача 6 лабы:

Как всегда дана СЛАУ вида $Ax = b$,

1. Построить СЛАУ 4-го порядка с плохо обусловленной матрицей ($\det A < 10^{-10}$).
2. Попытаться найти ее решение методом оптимального исключения (метод Гаусса-Жордано).
3. Нормализовать СЛАУ: $Ax = b \quad \rightarrow \quad (A^T A + \alpha E) x = A^T b$, где $\alpha$- параметр регуляризации.
4. Провести серию вычислений для определения нормального псевдорешения нормализованной системы.
5.  Найти оптимальное значение параметра $\alpha$, обеспечивающего минимальное значение сферической нормы невязки исходной системы для псевдорешения.

$$Ax = b \quad \rightarrow \quad (A^TA + \alpha E)x = A^Tb$$

$$\min_x \{F(x)\} = min_x \{(Ax - b, Ax - b) + \alpha (x, x)\} \geq 0$$

$$F(x) = (Ax, Ax) + 2 (Ax, b) + (b, b) + \alpha (x, x)$$

$$F(x) = (x, A^T Ax) + \alpha (x, x) + 2 (x, A^T b) + (b, b)$$

Берём дополнительным условием необходимое условие, чтобы найти минимум: $\frac{\displaystyle dF(x)}{\displaystyle dx} = 0$

$$\Rightarrow (A^T A + \alpha E)x = A^T b$$
$$x_{\alpha} = (A^T A + \alpha E)^{-1} A^T b$$

Соответственно подсчитываем невязку $\delta = b - A x_{\alpha}$ и её норму $\lVert \delta \rVert_2$.

In [171]:
import numpy as np
from typing import Tuple
from sympy import Matrix, init_printing
init_printing()

def GenMatrix(N: int) -> Tuple[np.ndarray, np.ndarray]:
  Mtrx = np.random.randint(20, size=(1, N + 1))
  cnt = 1

  while (cnt < N):
    if (cnt % 2 == 0):
      randRow = np.random.randint(20, size=(1, N + 1))
      Mtrx = np.concatenate((Mtrx, randRow), axis=0)
    else:
      Mtrx = np.concatenate((Mtrx, [Mtrx[-1, :] + 0.1]), axis=0)
    cnt += 1

  Mtrx = Mtrx.astype('float64')

  A = Mtrx[:, 0 : N]
  b = Mtrx[:, -1]
  b = b.reshape(N, 1)
  return (A, b)

def regularization(A: np.ndarray, b: np.ndarray, alpha: float) -> np.ndarray:
  x1 = b.copy()
  x1 = np.matmul(np.linalg.inv(np.matmul(A.T, A) + np.eye(b.shape[0]) * alpha), np.matmul(A.T, b))
  # display(Matrix(x1))
  return x1

def findMin(A: np.ndarray, b: np.ndarray) -> Tuple[np.ndarray, float]:
  dNorm: float = 50.0
  sol = []
  l = 0
  r = 10
  eps = 1e-5

  while (dNorm > eps):
    tl: float = l + (r - l) / 3
    tr: float = r - (r - l) / 3

    # Тернарный поиск
    xl = regularization(A, b, tl)
    dl = b - np.matmul(A, xl)

    xr = regularization(A, b, tr)
    dr = b - np.matmul(A, xr)

    if (np.linalg.norm(dr) > np.linalg.norm(dl)):
      r = tr
      dNorm = np.linalg.norm(dr)
      sol = xl.copy()
    else:
      l = tl
      dNorm = np.linalg.norm(dl)
      sol = xr.copy()

  return (sol, (l + r) / 2)

In [172]:
A, b = GenMatrix(4)

print("Сгенерированный столбец:")
display(Matrix(b))
print("Сгенерированная матрица:")
display(Matrix(A))
print("Определитель матрицы = ", np.linalg.det(A))

x, min_alpha = findMin(A, b)
d = b - np.matmul(A, x)

print("Найденное решение:")
display(Matrix(x))
print("Невязка:")
display(Matrix(d))
print("Сферическая норма невязки = ", np.linalg.norm(d))
print("Минимальное alpha = ", min_alpha)

Сгенерированный столбец:


⎡18.0⎤
⎢    ⎥
⎢18.1⎥
⎢    ⎥
⎢7.0 ⎥
⎢    ⎥
⎣7.1 ⎦

Сгенерированная матрица:


⎡7.0   16.0  12.0  9.0⎤
⎢                     ⎥
⎢7.1   16.1  12.1  9.1⎥
⎢                     ⎥
⎢12.0  19.0  8.0   2.0⎥
⎢                     ⎥
⎣12.1  19.1  8.1   2.1⎦

Определитель матрицы =  -1.2612133559741184e-14
Найденное решение:


⎡-1.06650173664093⎤
⎢                 ⎥
⎢0.643852472305298⎥
⎢                 ⎥
⎢0.786575794219971⎥
⎢                 ⎥
⎣0.636106729507446⎦

Невязка:


⎡2.50339508056641e-6 ⎤
⎢                    ⎥
⎢-8.22544098610933e-7⎥
⎢                    ⎥
⎢4.05311584472656e-6 ⎥
⎢                    ⎥
⎣7.27176665549223e-7 ⎦

Сферическая норма невязки =  4.888772819149664e-06
Минимальное alpha =  2.0097272630703612e-07
